# Perfect Solve of Puzzles using a SAT solver

Kaggle's [Conway's Reverse Game of Life 2020](https://www.kaggle.com/c/conways-reverse-game-of-life-2020/overview) can be viewed as a [boolean satisfiability problem](https://en.wikipedia.org/wiki/Boolean_satisfiability_problem).

In this notebook, we will:
 - presents the problem in a form solvable by SAT solver
 - use in a notebook the [kissat SAT solver](http://fmv.jku.at/kissat/), which won first place in the main track of the [SAT Competition 2020](https://satcompetition.github.io/2020) and first place on unsatisfiable instances.


In [ ]:
import os
import random
import subprocess
import time

import numpy as np
import pandas as pd

from joblib import Parallel, delayed
from scipy.signal import convolve2d
from sklearn.metrics import mean_absolute_error
from sympy.logic import POSform

In [ ]:
TIME_BUDGET = 8.9 * 3600  # Maximum time allowed for our notebook
MAX_BUDGET = 10  # Maximum time spent on a single puzzle
N = 25  # grid dimension
DELTAS=(1, )  # list of deltas we are trying to solve

### Install and compile kissat SAT solver from sources

In [ ]:
!git clone https://github.com/arminbiere/kissat.git >/dev/null
!cd kissat && ./configure && make all >/dev/null

### Generate CNF for one cell (live or dead)

To use a SAT solver we must express the problem in the [conjunctive normal form](https://en.wikipedia.org/wiki/Conjunctive_normal_form) (CNF).

The easiest way I found to generate the proble in CNF, is to create the truth table of the problem and use the [Quine–McCluskey algorithm](https://en.wikipedia.org/wiki/Quine%E2%80%93McCluskey_algorithm) to generate an optimal CNF.

We are lucky `sympy` implements it.


Following code will generate all cases of a 3x3 grid on which we apply the Game Of Life rules. Two lists keep results of combinations resulting in `live` or `dead`  cell. Each cell of the 3x3 grid is represented by a letter `a` to `i`:

In [ ]:
# a b c
# d e f
# g h i

live = []
dead = []
for a in (0,1):
    for b in (0,1):
        for c in (0,1):
            for d in (0,1):
                for e in (0,1):
                    for f in (0,1):
                        for g in (0,1):
                            for h in (0,1):
                                for i in (0,1):
                                    crown = (a+b+c+d+ f+g+h+i)
                                    var = [a, b, c, d, e, f, g, h, i]
                                    if crown == 3 or (e == 1 and crown == 2):
                                        live.append(var)
                                    else:
                                        dead.append(var)

live = POSform(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i'], live)
dead = POSform(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i'], dead)

Here is the CNF that a live cell shall validate:

In [ ]:
live

### CNF for the general case

`x` is the resulting cell.

In [ ]:
# x

# a b c
# d e f
# g h i

valid = []
for a in (0,1):
    for b in (0,1):
        for c in (0,1):
            for d in (0,1):
                for e in (0,1):
                    for f in (0,1):
                        for g in (0,1):
                            for h in (0,1):
                                for i in (0,1):
                                    crown = (a+b+c+d+ f+g+h+i)
                                    x = 1 if crown == 3 or (e == 1 and crown == 2) else 0
                                    var = [x, a, b, c, d, e, f, g, h, i]
                                    valid.append(var)

valid = POSform(['x', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i'], valid)

### Compute score of each puzzle

In [ ]:
def life_step(X):
    nbrs_count = convolve2d(X, np.ones((3, 3)), mode='same', boundary='wrap') - X
    return (nbrs_count == 3) | (X & (nbrs_count == 2))


def individual_scores(pred):
    scores = []
    for i in df.index:
        delta = df.loc[i][0]
        start = np.asarray(pred.loc[i]).reshape(N, N)
        end   = np.asarray(df.loc[i][1:]).reshape(N, N)
        x = start
        for _ in range(delta):
            x = life_step(x)
        scores.append(mean_absolute_error(x, end))
    return scores

### Turn into a template

In [ ]:
def templatize(cnf):
    return (
        str(cnf)
        .replace(' & ', ' 0\n')
        .replace(' | ', ' ')
        .replace('(', '')
        .replace(')', '')
        .replace('~', '-')
        + ' 0\n'
    )

In [ ]:
template_live = templatize(live)
template_dead = templatize(dead)
template_intermediate = templatize(valid)

In [ ]:
def replace(template, a, b, c, d, e, f, g, h, i, x=None):
    return (
        template
        .replace('a', str(a))
        .replace('b', str(b))
        .replace('c', str(c))
        .replace('d', str(d))
        .replace('e', str(e))
        .replace('f', str(f))
        .replace('g', str(g))
        .replace('h', str(h))
        .replace('i', str(i))
        .replace('x', str(x))
    )

In [ ]:
def wrap(val):
    return (val + N) % N

In [ ]:
# convert a position (x, y) at layed d to a variable number
def v(x, y, d):
    return 1 + wrap(x) + wrap(y) * N + d * N * N

In [ ]:
def get_clauses(puzzle, delta):
    clauses = ''
    
    for x in range(N):
        for y in range(N):

            clauses += replace(template_live if puzzle[x, y] else template_dead,
                            v(x-1, y-1, 0), v(x, y-1, 0), v(x+1, y-1, 0),
                            v(x-1, y  , 0), v(x, y  , 0), v(x+1, y  , 0),
                            v(x-1, y+1, 0), v(x, y+1, 0), v(x+1, y+1, 0))

    for d in range(1, delta):
        for x in range(N):
            for y in range(N):
                clauses += replace(template_intermediate,
                                v(x-1, y-1, d), v(x, y-1, d), v(x+1, y-1, d),
                                v(x-1, y  , d), v(x, y  , d), v(x+1, y  , d),
                                v(x-1, y+1, d), v(x, y+1, d), v(x+1, y+1, d),
                                v(x, y, d-1))
    return clauses

In [ ]:
def error(idx, pred):
    delta = df.loc[idx][0]
    end = np.asarray(df.loc[idx][1:]).reshape(N, N)

    x = pred
    for _ in range(delta):
        x = life_step(x)

    return mean_absolute_error(x, end)

### Solve one puzzle

In [ ]:
def solve(idx):
    budget = TIME_BUDGET - (time.time() - start_time)
    budget = min(budget, MAX_BUDGET)
    if budget <= 0:
        return None
    
    # load puzzle
    delta = df.loc[idx][0]
    puzzle = np.asarray(df.loc[idx][1:]).reshape(N,N)

    # get SAT clauses
    clauses = get_clauses(puzzle, delta)
    nb = clauses.count('\n')

    # write .cnf file
    filename = f'puzzles_cnf/{idx}.cnf'
    with open(filename, 'w') as f:
        f.write(f'c puzzle {idx}, delta {delta}\n')
        f.write(f'p cnf {N*N*delta} {nb}\n')
        f.write(clauses)
    
    # invoke kissat
    cmd = ["timeout", f'{budget}s', "./kissat/build/kissat", "-q", filename]
    cmd = subprocess.Popen(cmd, stdout=subprocess.PIPE)
    solution = cmd.communicate()[0]
    solution = solution.decode('utf8')
    os.remove(filename)
    
    if solution == '':
        print(idx, "timeout")
        return None
        
    print(idx, "solved")

    # parse solution
    parsed = []
    for x in solution.split():
        try:
            parsed.append(int(x))
        except:
            pass
    parsed = set(parsed)

    # format solution
    solution = np.zeros((N, N), dtype=int)
    for x in range(N):
        for y in range(N):
            z = v(x, y, delta-1)
            if z in parsed:
                solution[x, y] = 1

    return solution

### Load puzzles

In [ ]:
df = pd.read_csv('../input/conways-reverse-game-of-life-2020/test.csv', index_col='id')

### Use test.csv as baseline of our submission

In [ ]:
!mkdir -p puzzles_cnf
submission = df.copy()
submission.drop(['delta'], inplace=True, axis=1)
submission['score'] = individual_scores(submission)

### Order puzzles to solve easy ones first

In [ ]:
to_solve = list(set(df[df.delta.isin(DELTAS)].index).intersection(set(submission[submission.score > 0].index)))
to_solve = [x for _, x in sorted(zip(submission.loc[to_solve].score / df.loc[to_solve].delta, to_solve), reverse=True)]

### Run Jobs in Parallel

In [ ]:
start_time = time.time()

solutions = Parallel(n_jobs=-1)(
    delayed(solve)(i) for i in to_solve
)

print(f'Completely solved {sum([x is not None for x in solutions])} puzzles in {time.time() - start_time:.2f} seconds 🔥🔥🔥')

### Save submission

In [ ]:
submission.drop(['score'], axis=1, inplace=True)

for i, s in zip(to_solve, solutions):
    if s is not None:
        submission.loc[i] = s.reshape(N*N)

submission.rename(columns={f'stop_{x}': f'start_{x}' for x in range(N*N)}, inplace=True)
submission.to_csv(f'submission.csv')

# Conclusion

This approach helped to solve all puzzles with `delta` equal to 1 or 2.

I learnt a lot about SAT and CNF during this competition.

If you like this notebook, please leave a comment 🖊, upvote 👍, and put a smile on your face 😀.